In [4]:
import numpy as np
import pandas as pd

In [5]:
# Load data 
movies = pd.read_csv('movies.csv')
credits = pd.read_csv('credits.csv')

In [6]:
# merge databases
movies = movies.merge(credits,on='title')

In [7]:
# Remove unecessery column from data 
"""
imp attribues: (String Attribute)
genres,id,keywords,title,overview,cast,crew
"""

movies = movies[['movie_id','title','overview', 'genres','keywords','cast','crew']]

In [8]:
# Remove Null data 
movies.dropna(inplace=True)
movies.isnull().sum()
movies.duplicated().sum()

0

In [9]:
# manipulating genres column
import ast 


def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])

    return L

movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

In [10]:
def convertCast(obj):
    L = []
    count = 0
    for i in ast.literal_eval(obj):
        if count != 3:
            L.append(i['name'])
            count += 1
        else:
            break
    return L

movies['cast'] = movies['cast'].apply(convertCast)

In [11]:
# clean crew
def convertCrew(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] =='Director':
            L.append(i['name'])
            break
    return L

movies['crew'] = movies['crew'].apply(convertCrew)

In [12]:
# clean overview column
movies['overview'] = movies['overview'].apply(lambda x:x.split())


In [13]:
movies['geners'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x ] )
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x ] )
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in  x ] )
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in  x ] )

In [14]:
# create tag column 

movies['tag'] = movies['overview'] + movies['genres'] + movies['cast'] + movies['crew']

In [15]:
new_df = movies[['movie_id','title','tag']]

In [16]:
new_df['tag'] = new_df['tag'].apply(lambda x:" ".join(x))


C:\Users\shova\AppData\Local\Temp\ipykernel_1772\3243403044.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tag'] = new_df['tag'].apply(lambda x:" ".join(x))


In [17]:
new_df.head() 

,movie_id,title,tag
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [18]:
new_df['tag'] = new_df['tag'].apply(lambda x:x.lower())

C:\Users\shova\AppData\Local\Temp\ipykernel_1772\1436014886.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tag'] = new_df['tag'].apply(lambda x:x.lower())


# Vectorization
To make the prediction or recommandation 

Text -> vector (5000 vectors ) -> pick a vector (keyword,movie)  -> suggest 5 nearest vector (recommend)

To understand watch : https://www.youtube.com/watch?v=1xtrIEwY_zY
from 1.00.00


In [19]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words = 'english')

In [20]:
vectors = cv.fit_transform(new_df['tag']).toarray()

In [21]:
import nltk

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()


In [22]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    
    return y

In [23]:
new_df['tag'] = new_df['tag'].apply(stem)

C:\Users\shova\AppData\Local\Temp\ipykernel_1772\4074900442.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tag'] = new_df['tag'].apply(stem)


In [24]:
# Find similarities between movies 

from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

C:\Users\shova\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [25]:
def Recommend(movie):
    movie_index = new_df[new_df['title']==movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]

    for i in movie_list:
        print(new_df.iloc[i[0]].title)


In [26]:
Recommend("Batman Begins")

Amidst the Devil's Wings
The Dark Knight Rises
The Dark Knight
Batman
Dead Man Down


In [27]:
import pickle

In [28]:
pickle.dump(new_df.to_dict(),open('movies_dict.plk','wb'))

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))